In [16]:
"""
Eric Born
Class: CS677 - Summer 2
Date: 12 Aug 2019
Final project
Predicting the winning team in the video game League of Legends
"""

import os
from sys import exit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

sns.set_style("darkgrid")

# setup input directory and filename
data = 'games'
input_dir = r'C:\Users\TomBrody\Desktop\School\677\Final'
ticker_file = os.path.join(input_dir, data + '.csv')

# read csv file into dataframe
try:
    lol_df = pd.read_csv(ticker_file)
    print('opened file for ticker: ', data,'\n')

except Exception as e:
    print(e)
    exit('failed to read LoL data from: '+ str(data)+'.csv')

# describe the total rows and columns
print('The total length of the dataframe is', lol_df.shape[0], 'rows',
      'and the width is', lol_df.shape[1], 'columns')

opened file for ticker:  games 

The total length of the dataframe is 51490 rows and the width is 61 columns


In [17]:
# remove columns creationTime and seasonId
lol_df.drop(lol_df.columns[[1, 3]], 
                     axis = 1, inplace = True)

# describe the total rows and columns
print('The total length of the dataframe is', lol_df.shape[0], 'rows',
      'and the width is', lol_df.shape[1], 'columns')

The total length of the dataframe is 51490 rows and the width is 59 columns


In [20]:
# print the number of wins by team 1 and team 2 
for win in range(1, 3):
    print('Total team', win, 'wins:', sum(lol_df.winner == win))

# store wins in variable for each time
team1 = sum(lol_df.winner == 1)
team2 = sum(lol_df.winner == 2)

# create a ratio of the wins
ratio = round(team1 / team2, 4)
print('\nTeam win ratios\n','team 1 : team 2\n', str(ratio)+' :   1')

Total team 1 wins: 26077
Total team 2 wins: 25413

Team win ratios
 team 1 : team 2
 1.0261 :   1


In [ ]:
# split dataset into 33% test 66% training
lol_df_train, lol_df_test = train_test_split